## Nuvem de palavra: Planos de governo dos candidatos à Prefeitura de Fortaleza

Notebook das nuvens de palavras para a produção de reportagem para o jornal *Diário do Nordeste* sobre os termos que mais se repetem nas propostas de governo dos nove candidatos à Prefeitura de Fortaleza em 2024.

A análise aponta que saúde, segurança, educação e trabalho são os termos mais recorrentes. Saúde é a palavra mais citada, em 456 ocasiões. Segurança aparece em seguida, com 214. Ao lado, educação, com 189 menções, e cultura, com 130. Trabalho, inovação, meio ambiente, infraestrutura e renda surgem a seguir. Há ainda termos como vida e áreas, que aparecem de forma recorrente com diferentes aplicações, como "defesa da vida" e "qualidade de vida".

Os dados reforçam avaliações de cientistas políticos que pesquisam a conjuntura local. Para o levantamento, os termos vazios de significado para o plano de governo foram desconsiderados, como pronomes, artigos e preposições. Palavras como "Fortaleza", "política", "Prefeitura", "projeto" e "público" também foram retiradas para restringir o resultado apenas a propostas dos candidatos.


A matéria foi publicada no dia 24 de setembro de 2024 e pode ser lida [neste link](https://diariodonordeste.verdesmares.com.br/pontopoder/quais-palavras-mais-aparecem-nos-planos-de-governo-dos-candidatos-a-prefeitura-de-fortaleza-em-2024-1.3550684), no *Diário do Nordeste*.

**Coleta e análise de dados:** Igor Cavalcante

**Reportagem:** Igor Cavalcante

# Introdução

Esta análise está dividida em duas etapas. Na primeira, os dados são tratados, já que eles estão disponibilizados em formato .pdf. Portanto, é necessário transformá-los em .txt e limpá-los. 

Na segunda etapa, a nuvem de palavras será efetivamente produzida.

## Instale e importe as bibliotecas

In [ ]:
# Biblioteca para a criação das nuvens de palavras
!pip install wordcloud

# Biblioteca para manipular os arquivos .pdf
!pip install PyPDF2

In [2]:
import re # Manipulação de texto
from wordcloud import WordCloud, STOPWORDS # Criação das nuvens de palavras
import matplotlib.pyplot as plt # Visualização dos dados
from collections import Counter #Contagem de palavras
import pandas as pd # Manipulação e análise dos dados
import os # Acesso aos arquivos
from PyPDF2 import PdfReader

# Etapa 1: Preparação dos arquivos para análise

Os arquivos cadastrados pelos candidatos no sistema do Tribunal Superior Eleitoral (TSE) ficam registrados na página de cada concorrente ao Executivo a partir [deste link](https://divulgacandcontas.tse.jus.br/divulga/#/home). 

Os dados são cadastrados em .pdf, portanto, é preciso converter cada um em arquivos .txt.

Em seguida, é preciso criar um arquivo reunindo todas as propostas.

Obs.: Os candidatos Técio Nunes e André Fernandes apresentaram, inicialmente, uma prévia do plano de governo e, só depois, o documento definitivo. Portanto, os arquivos dos dois foram atualizados. 

In [9]:
# Definir o diretório onde os .pdfs estão localizados e onde os .txts serão salvos
base_dir = os.path.dirname(os.path.abspath('nuvem_de_palavras'))
pasta_pdf = os.path.join(base_dir, 'planos_pdf/')
pasta_txt = os.path.join(base_dir, 'planos_txt/')

# Lista dos arquivos .pdf
arquivos_pdf = [
    'andre_fernandes_atualizado.pdf', 'capitao_wagner.pdf', 'chico_malta.pdf',
    'eduardo_girao.pdf', 'evandro_leitao.pdf', 'george_lima.pdf',
    'jose_sarto.pdf', 'tecio_nunes_atualizado.pdf', 'ze_batista.pdf'
]

## Conversão e limpeza dos arquivos

In [11]:
# Ao converter o .pdf surgem alguns caracteres especiais e quebras de linha. Nos planos de governos há ainda letras maiúsculas e minúsculas que não são relevantes aqui, portanto, essa função de limpeza vai remover as quebras de linha, os caracteres especiais, os múltiplos espaços e deixar todos os textos em caixa baixa
def limpeza(text):
    text = re.sub(r'[\r\n]+', ' ', text) # Substitui as quebras de linha por espaço
    text = re.sub(r'[^\w\s]', '', text) # Remove caracteres especiais e pontuações, deixando apenas letras, números e espaços
    text = re.sub(r'\s+', ' ', text) # Remove múltiplos espaços
    return text.lower().strip() # Deixa o texto em caixa baixa

# Aqui, cada arquivo .pdf será acessado, seu texto extraído na íntegra a cada página e, em seguida, tudo será concatenado como um só texto
for arquivo_pdf in arquivos_pdf:
    pdf_path = os.path.join(pasta_pdf, arquivo_pdf)
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text() if page.extract_text() else ""

# Cada texto será submetido à função de limpeza, removendo o que foi citado anteriormente
    text_limpo = limpeza(text)

# Depois de limpo, o texto é salvo em um arquivo .txt em uma pasta exlusiva para ele
    arquivo_txt = arquivo_pdf.replace('.pdf', '.txt')
    txt_path = os.path.join(pasta_txt, arquivo_txt)
    with open(txt_path, 'w', encoding='utf-8') as f:
        f.write(text_limpo)

# Com o arquivo de cada candidato gerado, agora é produzido um novo arquivo compilando das propostas de todos. Isso será útil para produzir uma nuvem com os temos mais citados por todos os planos
with open(os.path.join(pasta_txt, 'arquivo_final.txt'), 'w', encoding='utf-8') as output_file:
    for arquivo_pdf in arquivos_pdf:
        arquivo_txt = arquivo_pdf.replace('.pdf', '.txt')
        txt_path = os.path.join(pasta_txt, arquivo_txt)
        
        with open(txt_path, 'r', encoding='utf-8') as f:
            texto = f.read()
            output_file.write(texto + '\n')

# Etapa 2: Nuvem de palavras

A partir daqui, a nuvem de palavras será efetivamente criada. 

Os textos tratados anteriormente passarão por uma segunda "limpeza" para excluir as palavras vazias de sentido. Neste caso, além das stopwords da biblioteca, também há termos específicos, adicionados manualmente.

In [12]:
# Definir o diretório onde os .txts estão localizados e onde as nuvens de palavras serão salvas
diretorio = os.path.join(base_dir, 'planos_txt/')
arquivos_gerados = os.path.join(base_dir, 'arquivos_gerados/')

# Lista dos arquivos .txts
arquivos = [
    'andre_fernandes_atualizado.txt', 'capitao_wagner.txt', 'chico_malta.txt',
    'eduardo_girao.txt', 'evandro_leitao.txt', 'george_lima.txt',
    'jose_sarto.txt', 'tecio_nunes_atualizado.txt', 'ze_batista.txt', 'arquivo_final.txt'
]

## Removendo as palavras vazias de sentido

In [13]:
# A função gera uma nuvem de palavras a partir do texto fornecido, excluindo palavras que são irrelevantes para a análise
def word_cloud(texto_limpo, palavras_vazias):
    stopwords = set(STOPWORDS)
    stopwords.update(palavras_vazias)
    wordcloud = WordCloud(width=800, height=400, max_words=50, background_color='white', stopwords=stopwords).generate(texto_limpo)
    return wordcloud

# Palavras acrescentadas manualmente que não têm significado relevante nas propostas de governo
palavras_vazias = {"a", "à", "á", "ã", "ao", "é", "ainda", "além", "alguma", "algumas", "algum", "alguns", "ampla", "amplas", "amplo", "amplos", "ante", "após", "até", "através", "cada", "coisa", "coisas", "com", "como", "pra", "dá", "fazer", "garantir", "contra", "contudo", "da", "das", "de", "dela", "delas", "dele", "deles", "depois", "serão", "recurso", "dessa", "dessas", "desse", "desses", "desta", "destas", "deste", "destes", "disso", "disto", "do", "dos", "durante", "e", "ela", "elas", "ele", "eles", "em", "então", "entre", "era", "essa", "essas", "esse", "esses", "esta", "estas", "este", "estes", "eu", "foi", "foram", "há", "isso", "isto", "já", "la", "lá", "lhe", "tenham", "lhes", "lo", "mas", "me", "mesma", "mesmas", "mesmo", "mesmos", "meu", "meus", "minha", "minhas", "muita", "muitas", "muito", "muitos", "na", "nas", "nem", "no", "nos", "nós", "nossa", "nossas", "nosso", "nossos", "num", "numa", "o", "os", "ou", "onde", "para", "pela", "pelas", "pelo", "pelos", "por", "qual", "quando", "que", "assim", "quem", "se", "seja", "sem", "seu", "seus", "só", "sob", "sobre", "sua", "suas", "tal", "também", "te", "tem", "tendo", "ter", "teu", "teus", "toda", "todas", "todo", "todos", "tu", "tua", "tuas", "tudo", "um", "uma", "você", "vocês", "vos", "vosso", "vossos", "será", "Fortaleza", "programa", "na", "proposta", "garantindo", "cidade", "se", "governo", "população", "página", "pessoa", "mais", "política", "promover", "pela", "forma", "municipal", "plano", "gestão", "acesso", "pessoas", "sistema", "serviço", "não","questões", "partir", "melhoria", "espaço", "necessidades", "103", "aos", "às", "são", "área", "eixo", "políticas", "Prefeitura", "Nova", "criação", "rua", "sociedade", "público", "atualizado", "primeiro", "aquilo", "desde", "pública", "município", "fim", "está", "s", "documento","sendo", "estamos", "fundamental", "continuamente", "eleitoral", "apenas", "vez", "hoje", "proposições", "objetivo", "criar", "municipais", "fortalezense", "vamos", "equipe", "tema", "ideias", "possam", "criando", "novas", "meio", "apenas", "deve", "feita", "sejam", "ser", "mil", "outros", "seguirá", "desenvolvimento", "tratar", "inicial", "receber", "grande", "diversas", "temos", "serviços", "atendimento", "qualidade", "comunidade", "social", "grande", "atendimento", "lima", "ampliar", "implementar", "grupo", "cidadão", "projeto", "propostas", "fortalecer", "participação", "construir", "direito", "povo", "processo", "eixos", "redução", "fato", "realmente", "públicas", "públicos", "capital", "1", "2", "3", "4", "5", "6", "7", "8", "9", "0", "20252028", "ii"}

## Gerando a nuvem de palavras de cada candidato

Crie o loop para gerar nuvens de palavras para cada candidato específico e mais uma considerando todos os postulantes.

In [ ]:
# Aqui, cada arquivo será acessado, seu texto será lido, as palavras vazias de sentido ignoradas e a nuvem de palavras será gerada a partir disso
for arquivo in arquivos:
    caminho_completo = os.path.join(diretorio, arquivo)
    with open(caminho_completo, 'r', encoding='utf-8') as f:
        text = f.read()
    texto_limpo = limpeza(text)
    nuvem = word_cloud(texto_limpo, palavras_vazias)
    
# Gerando graficamente a nuvem de palavras
    plt.figure(figsize=(15, 5))
    plt.imshow(nuvem, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"Nuvem de Palavras - {arquivo.split('.')[0].replace('_', ' ').title()}")
    plt.show()

# Salvando as nuvens de palavras como imagens
    nuvem.to_file(os.path.join(arquivos_gerados, f"nuvem_{arquivo.split('.')[0]}.png"))

## Lista de palavras

In [15]:
# Gerando um arquivo com os termos mais citados que aparecem na nuvem e a quantidade de repetição
def gerar_ranking_palavras(arquivos, diretorio, palavras_vazias):
    dfs = []
    stopwords = set(STOPWORDS)
    stopwords.update(palavras_vazias)
    for arquivo in arquivos:
        caminho_completo = os.path.join(diretorio, arquivo)
        with open(caminho_completo, 'r', encoding='utf-8') as f:
            text = f.read()
        texto_limpo = limpeza(texto)
        palavras = texto_limpo.split()
        palavras_filtradas = [palavra for palavra in palavras if palavra.lower() not in stopwords]
        contador = Counter(palavras_filtradas)
        df_ranking = pd.DataFrame(contador.most_common(50), columns=[f'Palavra_{arquivo.split(".")[0]}', f'Frequência_{arquivo.split(".")[0]}'])
        dfs.append(df_ranking)

    # Concatenando os DataFrames lado a lado
    df_final = pd.concat(dfs, axis=1)
    return df_final

# Gerando o ranking das palavras mais usadas por cada candidato
df_ranking_palavras = gerar_ranking_palavras(arquivos, diretorio, palavras_vazias)

# Salvando a tabela como .csv
df_ranking_palavras.to_csv(os.path.join(arquivos_gerados, 'ranking_palavras_candidatos.csv'), index=False, encoding='utf-8')
